In [1]:
import re
import os
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import urllib 

# 解析

In [28]:
def recursive_parser_medicines(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
            
            content = child.text.strip()
            if len(content) == 0:
                continue
                
                
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['mediaPrmo ss']:
                    continue
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['credits']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe']: # 加视频
                continue

            if tag == 'center':
                texts.append(content)
                continue


            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'app-brightcove-video', 'nhsuk-body-s nhsuk-u-margin-top-7 nhsuk-u-secondary-text-color']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser_medicines(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser_medicines(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [29]:
def recursive_parser_health(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
            
            content = child.text.strip()
            if len(content) == 0:
                continue
            
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe', 'nav']: 
                continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'app-brightcove-video', 'nhsuk-body-s nhsuk-u-margin-top-7 nhsuk-u-secondary-text-color']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser_health(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser_health(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [30]:
def recursive_parser_mis(item, texts, pre_class):
    for child in item:
        try:

            tag = child.name
            
            content = child.text.strip()
            if len(content) == 0:
                continue
                
                
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['mediaPrmo ss']:
                    continue
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['credits']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe']: # 加视频
                continue

            if tag == 'center':
                texts.append(content)
                continue

            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'app-brightcove-video', 'nhsuk-body-s nhsuk-u-margin-top-7 nhsuk-u-secondary-text-color']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser_medicines(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser_medicines(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [31]:
def parser(input_path, output_path, tag, results):
    for root, dirs, file_names in os.walk(input_path):
        """
        os.walk()
        root 所指的是当前正在遍历的这个文件夹的本身的地址
        files_names同样是 list , 内容是该文件夹中所有的文件(不包括子目录)
        函数会自动改变root的值使得遍历所有的子文件夹。
        所以返回的三元元组的个数为所有子文件夹（包括子子文件夹，子子子文件夹等等）加上1（根文件夹）
        """
        
        for file_name in file_names:
            html_file = os.path.join(root, file_name)
            html_content = open(html_file, 'r', encoding='utf-8').read()
            selector = Selector(text=html_content)
#             try:
#                 title = selector.xpath("//h1//text()").extract()[0].strip()
#             except Exception as e:
#                 print(e)
#                 print(html_file)
#                 continue
            
            title = file_name.strip('.html')
            
            texts = []
            
            soup = BeautifulSoup(html_content, 'html.parser') 
            
            if tag=='health':
                item = soup.find('article')
                if item is None:
                    print(html_file)
                    continue
                else:
                    recursive_parser_health(item, texts, "")  
                    content = '\n'.join(texts)
                    if content == "":
                        continue
                    tmp = {
                        "title": title,
                        "content": content
                    }
                    results.append(tmp)
                
            if tag=='medicines':
                item = soup.find('article')
                if item is None:
                    print(html_file)
                    continue
                else:
                    recursive_parser_medicines(item, texts, "")
                    content = '\n'.join(texts)
                    if content == "":
                        continue
                    tmp = {
                        "title": title,
                        "content": content
                    }
                    results.append(tmp)
            
            if tag=='mis':
                abstract = soup.find('div',{'class':'nhsuk-u-margin-bottom-8'})
                if abstract is None:
                    continue
                item = soup.find('article')
                if item is None:
                    print(html_file)
                    continue
                else:
                    recursive_parser_mis(item, texts, "")
                    content = '\n'.join(texts)
                    if content == "":
                        continue
                    tmp = {
                        "title": title,
                        "content": content
                    }
                    results.append(tmp)

In [13]:
tags = ['health', 'medicines']
output_path = 'D:/Intern_Project/7_nhs/'
for tag in tags: 
    results = []
    mislist = []
    print(tag)
    source_path = f'D:/Intern_Project/7_nhs/{tag}_html'
    parser(source_path, output_path, tag, results, mislist)
    with open(f'D:/Intern_Project/7_nhs/{tag}_parser_results.json', 'w', encoding="utf-8") as f:
        f.write(json.dumps(results, indent=1, ensure_ascii=False))
        # f.write('\n'.join(results))

health
medicines
D:/Intern_Project/7_nhs/medicines_html\indapamide：how-and-when-to-take-indapamide：.html


In [47]:
tag = 'medicines'
source_path = f'D:/Intern_Project/7_nhs/{tag}_html'

results = []

parser(source_path, output_path, tag, results)

D:/Intern_Project/7_nhs/medicines_html\indapamide：how-and-when-to-take-indapamide：.html


In [48]:
tag = 'mis'
source_path = f'D:/Intern_Project/7_nhs/medicineslist_html/'

mis_results = []

parser(source_path, output_path, tag, mis_results)

In [50]:
print(len(results))
print(len(mis_results))

1257
85


In [51]:
results.extend(mis_results)

In [52]:
print(len(results))

1342


In [53]:
with open(f'D:/Intern_Project/7_nhs/all_medicines_parser_results.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(results, indent=1, ensure_ascii=False))

In [17]:
with open('D:/Intern_Project/7_nhs/health_parser_results.json', 'r', encoding='utf-8') as f:
    health_parser_results = json.load(f)
print(len(health_parser_results))

1226


In [19]:
for item in health_parser_results:
    item['content'] = item['content'].split("Further information")[0].split("Page last reviewed")[0]

In [23]:
print(health_parser_results[3])

{'title': 'Abdominal aortic aneurys', 'content': "An abdominal aortic aneurysm (AAA) is a swelling in the aorta, the artery that carries blood from the heart to the tummy. Most aneurysms do not cause any problems, but they can be serious because there's a risk they could burst (rupture).\nSymptoms of abdominal aortic aneurysm\nAbdominal aortic aneurysm often has no symptoms.\nYou usually only find out you have one during an abdominal aortic aneurysm screening test or during tests for another condition.\nIf an aneurysm gets bigger, you might sometimes notice:\ntummy or back pain\na pulsing feeling in your tummy\nNon-urgent advice:  See a GP if:\nyou have tummy or back pain that does not go away or keeps coming back\nyou feel a lump in your tummy\nThese symptoms can be caused by lots of things and do not mean you have an abdominal aortic aneurysm, but it's best to get them checked.\nImmediate action required: Call 999 if you or someone else:\nhave sudden, severe pain in your tummy or bac